In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd
import re #정규식을 쓰기위해서 필요한 패키지
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-9ea9960b25f5>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
TARGET_URL = "https://news.joins.com/Search/totalnews?page=&Keyword=%EC%BD%94%EB%A1%9C%EB%82%98&PeriodType=OneMonth&SortType=New&SourceGroupType=Joongang&SearchCategoryType=TotalNews"

TITLE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
DATE_OF_ARTICLE = []

In [3]:
def get_link_from_news_title(page_num, URL):
    for i in range(page_num): 
        current_page_num = 1 + i
        position = TARGET_URL.index('page=') 
        URL_with_page_num = TARGET_URL[:position+5] + str(current_page_num) + TARGET_URL[position+5:] 
        source_code_form_url = urllib.request.urlopen(URL_with_page_num)
        soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
        for title in soup.select('h2.headline.mg'):
            url_link = title.select('a')
            article_URL = url_link[0]['href'] 
            get_text(article_URL)

In [ ]:
#def get_text(URL):
#    source_code_form_url = urllib.request.urlopen(URL)
#    soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
#    title_link = soup.select('#article_title') 
#    TITLE_OF_ARTICLE.append(title_link[0].string)
#    date_url = soup.select('div.byline') 
#    date_of_article = date_url[0].select('em')
#    DATE_OF_ARTICLE.append(date_of_article[1].string)
#    content = soup.select('div.article_body')
#    for item in content:
#        string_item = str(item.find_all(text=True))
#        CONTENT_OF_ARTICLE.append(string_item)

def get_text(URL):  
    source_code_form_url = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
    title_link = soup.select('#article_title') 
    TITLE_OF_ARTICLE.append(title_link[0].string)
    date_url = soup.select('div.byline') 
    date_of_article = date_url[0].select('em')
    DATE_OF_ARTICLE.append(date_of_article[1].string)
    content = soup.select('div.article_body')
#    contentp = soup.find_all("p")
    contentimg = soup.find_all("img")

    str_contentimg =[]
    for div in soup.find_all('div','image'):
        img = div.find('img', alt=True)
        str_contentimg.append(img['alt'])

    for item in content:
            string_item = str(item.find_all(text=True))
#            for i in range(len(contentp)):
#                string_item = string_item.replace(str(contentp[i]),"")
            for i in range(len(str_contentimg)):
                string_item = string_item.replace(str(str_contentimg[i]),"")

    CONTENT_OF_ARTICLE.append(string_item)

In [ ]:
def get_total_num_of_article(URL):
    source_code_from_url = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    total_num_of_article = soup.select('span.total_number')
    num = total_num_of_article[0].string
    position = num.index('/') # 1-72/725건이라면, 725건만 가지고 검색하겠다. 나머지는 버리겠다는 의미
    num = num[position:]  # position인 '/' 오른쪽에 있는 부분만 남김 
    num = re.sub('[^0-9]', '', str(num)) # 0-9사이 숫자아닌것 모두 제거. 725 만 남음
    num = int(num)
    if(num % 10 >= 1): # 725을 10으로 나눴을때 나머지가 1보다 크거나 같으면
        page_num = int((num / 10) + 1) #그럴경우 10으로 나눠 1을 더함. 73가 됨
    else: 
        page_num = int(num / 10) # 그렇지 않을 경우는 그냥 10으로 나눠, 71이될거임
    return page_num

page_num = get_total_num_of_article(TARGET_URL)

In [6]:
get_link_from_news_title(1, TARGET_URL)

In [7]:
df = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE, DATE_OF_ARTICLE)), columns =['Title', 'Content', 'Date']) 

In [8]:
df['Date'] = df['Date'].str.replace('[^0-9]', '', regex=True)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d%H%M')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('.', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.replace('아티클 공통 : DA 250', '') 
df['Content'] = df['Content'].str.replace('아티클 공통 : 관련기사', '')
df['Content'] = df['Content'].str.lstrip()
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')
df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','')
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')


In [9]:
df.head()

,Title,Content,Date,Year,Month,Day
0,중국 한발 빨랐다유럽보다 먼저 백신여권 출시,중국이 전 세계에서 처음으로 백신 여권을 내놨다 왕이 중국 외교 담당 국무위원 겸 외교부장이 국제적 백신여권 도입 논의를 주도하겠다는 뜻 밝힌 지 하루 만이다9일 환구시보에 따르면 중국 외교부는 전날 채팅 애플리케이션인 위챗 미니프로그램을 통해 중국판 백신 여권인 국제여행 건강증명서를 출시했다고 밝혔다백신 여권은 신종 코로나바이러스 감염증 백신 접종 여부를 인증하는 일종의 증명서다 이 증명서를 제시하면 해외 입출국 과정 때 격리를 면제해준다증명서에는 코로나19 백신 접종 날짜와 백신 종류 핵산검사 결과 혈청 항체 형성 유무 등의 정보가 담긴다 해당 정보는 암호화한 QR코드를 통해 확인할 수 있고 디지털 버전 외에 종이로도 출력할 수 있다중국 외교부는 중국의 국제여행 건강증명서가 국가 간 건강하고 안전하며 질서 있는 인적 왕래를 추진하는 데 큰 역할을 할 것이라며 많은 국가와 중국이 건강증명 상호인증에 합의해 나가길 바란다고 밝혔다앞서 지난 7일 왕 부장은 기자회견에서 국제적인 백신여권 도입 논의를 주도하겠다는 뜻을 내비쳤다그는 국제 여행객을 위한 중국 버전의 전자 건강증명을 만들겠다면서 백신 접종 정보의 상호 인증을 실현해 안전하고 질서 있는 인적 왕래를 돕겠다고 말했다백신 여권 도입 논의는 앞서 유럽연합 등 서구권에서 먼저 시작됐다 그러나 중국이 발 빠르게 움직여 세계 최초 백신 여권 출시라는 타이틀을 얻은 셈이다 이를 통해 코로나19 사태에 국제 사회 인적 교류 주도권을 잡겠다는 뜻으로 해석된다다만 국제 사회가 여전히 백신 여권을 부정적으로 평가하는 만큼 상용화까지는 시간이 걸릴 것으로 보인다가오푸 중국 질병예방통제센터 주임은 국가 간 백신 접종 상호인증이 복잡한 문제라며 오랜 협상이 필요할 것으로 예측했다 관련기사 평화유지군올림픽 선수에도 제공 백신 주도권 잡기 나선 中 이민정 기자,2021-03-09 22:16:00,2021,3,9
1,태어난 지 37일 된 신생아 코로나19로 숨져그리스 애도,태어난 지 37일밖에 되지 않은 신생아가 신종 코로나바이러스 감염증으로 숨져 그리스가 충격에 빠졌다8일 AFP통신 등에 따르면 코 염증과 열 등의 증상을 보인 이 아이는 아테네의 한 어린이병원에서 치료를 받다가 지난 7일 끝내 숨졌다이는 그리스에서 발생한 코로나19 사망자 중 가장 어린 사례다 AFP통신은 그리스 전역에서 아이의 사망을 애도했다고 보도했다 이날 기준 그리스에서는 20만6281명의 코로나19 누적 확진자가 발생했으며 사망자는 6800명에 가까운 것으로 알려졌다키리아코스 미초타키스 그리스 총리는 소셜네트워크서비스를 통해 슬프게도 우리나라에서 가장 어린 코로나19 희생자가 나왔다며 생후 37일 중 17일을 코로나19와 싸웠다며 애도의 뜻을 전했다나운채 기자,2021-03-09 21:54:00,2021,3,9
2,접종 10시간 뒤 38도 이상 고열AZ백신 후유증 심했다,지난 4일 아스트라제네카 백신 주사를 맞고 10시간이 지난 뒤에 38도 이상의 고열과 오한 두통에 시달렸다 사실 나보다 하루 전에 백신을 맞은 의사가 후유증으로 응급실에 실려 갔다고 해서 마음의 준비를 했는데도 생각보다도 심하게 왔다AZ 신종 코로나바이러스 감염증 백신 접종을 마친 현직 의사가 백신 접종으로 인한 후유증이 생각보다 굉장히 심각했다고 털어놓았다9일 재활의학과 김경렬 전문의가 운영하는 유튜브 채널에는 아스트라제네카 백신 접종 솔직한 부작용 후기라는 제목의 영상이 올라왔다 김 전문의는새벽 1시 정도에 오한 때문에 잠에서 깼다 제일 높았던 체온은 387도였다며 고열인 상황에서 열을 재는 영상을 공개했다이 영상에서 김 전문의의 체온계는 37도와 38도 사이를 오갔다 김 전문의는 약을 먹으니 좀 나아졌다며 세 가지 약을 준비했는데 개인적으로 타이레놀이 제일 효과가 있었다고 말했다주변인 30명 중 70%는 백신 부작용 겪어자신의 주변 의사들과 환자들에게 백신 부작용 사례를 물은 결과도 공유했다 김 전문의는 약 먹고 출근하고서 다른 친구들에게 물었는데 30명 중 70%는 부작용이 있었고 60%는 나랑 비슷한 증상을 겪었다고 하더라며 일반 독감보다는 증상이 심했던 거 같다고 전했다 또 특히 20대 초반에서 40대 초반 젊은 분들이 심한 증상을 호소했고 50대 이상인 분들은 의외로 별다른 증상 없이 넘어간 분들이 많았다고 말했다 다만 젊다고 다 부작용 생긴 건 아니었고 20대 2명은 부작용이 없었다고 말했다김 전문의는 개인적으로 이번 주부터 AZ 백신 접종이 본격적으로 시작되면 많은 분이 심한 독감 유사 증상을 호소하실 것으로 보인다며 백신에 대한 거부감도 상당할 것이라고 말했다 또 백신 효과가 몇 년 이상 가는 게 아니다 보니 매년 이런 부작용을 겪으며 백신을 맞아야 한다면 대부분 납득하지 못할 거라고 생각한다고 말했다 집단면역 형성해야2차 접종 피하지 않겠다하지만 김 전문의는 그럼에도 불구하고 저는 지금 건강하신 분들이 AZ 백신 접종을 해서 집단면역을 형성해야 한다고 생각한다며 우리나라가 가진 백신은 이것밖에 없다 나도 2차 접종을 피하지 않겠다 백신을 계속 맞으면 부작용도 줄 것이라고 말했다다만 심장병 같은 기저질환 환자에게 접종하는 건 안전성 확인 뒤에 하는 게 좋겠다며 지금까지 사망한 사람은 AZ 백신 맞은 기저질환 있는 환자들이기 때문이라고 말했다끝으로 오늘 제가 한 이야기는 과학적 근거는 없는 개인적인 후기일 뿐이라고 덧붙였다영상을 본 네티즌들은 후기영상 감사하다 70% 정도 부작용 사례가 있다니 긴장도 된다 58세 여성인데 꼬박 2박 3일은 고생하고 나아지는 것 같다 타이레놀도 잘 안 든다 저희 병원에 있는 다른 직원들은 해열제를 3번 먹고도 약효가 떨어지면 또 열이 올라가 힘들었다는 댓글을 달았다신혜연 기자,2021-03-09 21:51:00,2021,3,9
3,정의용 인도 외교장관과 통화 국제사회 코로나 극복위해 노력,정의용 외교부 장관이 9일 수브라마냠자이샨카르 인도 외교장관과 통화를 갖고 한국-인도 양자관계 신종 코로나바이러스 감염증 대응 등에 대해 의견을 교환했다정 장관은 이날 취임 후 처음으로 자이샨카르 장관과 통화를 갖고 상황이 개선되는 대로 외교장관 간의 공동위 개최 외교국방 차관회의 등 고위급 교류 재개 방안을 적극 추진하기로 했다아울러 국제사회의 코로나 극복을 위해 함께 노력해나가기로 했다양 장관은 문재인 대통령과 나렌드라모다 총리 간의 각별한 우의와 신뢰를 바탕으로 최상의 특별전략적 동반자 관계를 유지해오고 있음을 평가하고 코로나 상황에서도 긴밀히 소통하면서 실질 협력을 강화해 나가기로 했다정 장관은 한반도 평화 구축을 위한 우리 정부의 노력을 설명했으며 자이샨카르 장관은 2021~2022년 유엔 안전보장이사회 비상임이사국으로서 한반도 평화 프로세스의 실질적 진전을 지원하겠다고 밝혔다홍수민 기자,2021-03-09 21:38:00,2021,3,9
4,간호대 5명 중 1명 남학생 군입대로 경력 단절 공중보건간호사 도입해야,간호대 남학생이 매년 크게 늘어나는 한편 의대치대한의대는 여학생이 급증하고 있는 것으로 나타났다 9일 대한간호협회가 보건복지부 면허관리정보시스템과 교육부 2020 교육통계연보를 분석한 결과에 따르면 간호대 남학생과 간호사 국시 합격자는 매년 늘어나고 있다 2019년 현재 재학 중인 간호대의 남학생 수는 2만4058명으로 전체 간호대 재학생의 224%에 달했다 2019년까지 배출된 남성 간호사는 전체 간호사 면허자의 43%인 1만7863명이었다 간호사 국시 남성 합격자 수도 매년 크게 늘었다 1996년 19명에서 2000년에는 45명으로 늘어났고 2009년 619명 2019년에는 2843명으로 급증했다 2019년 간호사 국시 합격자 8명 중 1명은 남자였다간호협회에 따르면 우리나라 최초의 남자 간호사는 1962년 면허를 취득한 조상문씨다 이후 2019년까지 모두 1만7863명의 남자 간호사가 배출됐다 간호대의 남학생 비율이 높아진처럼 의대의 여학생 비율도 높아졌다 2019년 기준 의과대 여학생 비율은 356%다 2019년까지 배출된 전체 의사 면허자 중 여성 비율은 261%인 점을 고려하면 앞으로 

In [11]:
df.to_csv("3주차 실습.csv",index=True, encoding='utf-8-sig')

In [74]:

TITLE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
DATE_OF_ARTICLE = []

URL = "https://news.joins.com/article/24008174"
 
y= []

x = soup.find_all('ul','text_type')
for i in x:
    y.append(i.get_text())
             
source_code_form_url = urllib.request.urlopen(URL)
soup = BeautifulSoup(source_code_form_url, 'lxml', from_encoding='utf-8')
title_link = soup.select('#article_title') 
TITLE_OF_ARTICLE.append(title_link[0].string)
date_url = soup.select('div.byline') 
date_of_article = date_url[0].select('em')
DATE_OF_ARTICLE.append(date_of_article[1].string)
content = soup.select('div.article_body')
contentp = soup.find_all("p")
contentimg = soup.find_all("img")

text_contentrelated = []
str_contentimg =[]
for div in soup.find_all('div','image'):
    img = div.find('img', alt=True)
    str_contentimg.append(img['alt'])

for item in content:
        string_item = str(item.find_all(text=True))
        for i in range(len(contentp)):
            string_item = string_item.replace(str(contentp[i]),"")
        for i in range(len(str_contentimg)):
            string_item = string_item.replace(str(str_contentimg[i]),"")
        for i in range(len(text_contentrelated)):
            string_item = string_item.replace(str(text_contentrelated[i]),"")

CONTENT_OF_ARTICLE.append(string_item)
print(CONTENT_OF_ARTICLE)


['[\'\\n\', \' \', \' \', \' \', \' \', \' \', \'\', \' \', \' \', \'  “너무 많아서 지금 떠오르지 않을 정도다. (당대표는) 불가피한 선택이었다.”  \', \'   당권·대권 분리 규정에 따라 차기 대선 1년 전인 9일 대표직에서 물러난\\xa0이낙연 더불어민주당 대표가 퇴임 기자 간담회에서 한 말이다. 임기 중 가장 후회되는 순간을 묻는 질문 등에 이런 답변이 나왔다.\\xa0\', \' \', \'   \\xa0  \', \'   대표직을 떠난 그는 대선 판도에 작지 않은 영향을 미칠 것으로 예상되는 4·7 재·보궐 선거의 상임선대위원장을 맡게 된다. 그리고 이 선거를 치른 뒤엔 대선 주자로서의 행보를 본격화할 예정이다.\\xa0\', \' \', \'   \\xa0  \', \'   7개월간의 당 대표 기간 동안 그는 한때 40%에 달했던 대선 후보 지지율이 10%대로 곤두박질치는 경험을 했다. \', \'자신에겐 야심작이었을지 몰라도 전직 대통령 사면 건의는 지지율 추락의 주범이었다. \', \'그는 이날 사면 건의에 대해 “언젠가 해야 할 과제라고 생각했었다. 국민 마음을 좀 더 세밀하게 헤아려야 한다는 아픈 공부가 됐다”고 했다. 또 지지율 하락에 대해선 “제 부족함과 정치의 어려움 때문”이라고 말했다.  \', \'   \\xa0  \', \' \', \' \', \' \', \' \', \' \', \' \', \'\', \' \', \'        \\xa0  \', \'   최측근이 스스로 목숨을 끊는 일도 있었다. 이런 우여곡절 끝에 대표직을 떠나는 그에게 서울·부산 보궐선거는 작은 반전의 계기가 될 수도 있다. \', \'오는 5월 전당대회 때까지 김태년 원내대표가 권한대행으로 당을 이끌지만, 이번 재·보선은 상임선대위원장으로 치르는 ‘이낙연의 선거’란 관측이 많다.\', \'\\xa0박원순·오거돈 전직 시장 때문에 치러지는 선거임에도 지난해 11월 당헌·당규 개정과 공천 강행을

In [31]:
for div in soup.find_all('div','image'):
    img = div.find('img', alt=True)
    str_contentimg.append(img['alt'])

for item in content:
        string_item = str(item.find_all(text=True))
        for i in range(len(contentp)):
            string_item = string_item.replace(str(contentp[i]),"")
        for i in range(len(str_contentimg)):
            string_item = string_item.replace(str(str_contentimg[i]),"")
        for i in range(len(text_contentrelated)):
            string_item = string_item.replace(str(text_contentrelated[i]),"")

CONTENT_OF_ARTICLE.append(string_item)
print(CONTENT_OF_ARTICLE)

['[\'\\n\', \' \', \' \', \' \', \' \', \' \', \'\', \' \', \' \', \'  “너무 많아서 지금 떠오르지 않을 정도다. (당대표는) 불가피한 선택이었다.”  \', \'   당권·대권 분리 규정에 따라 차기 대선 1년 전인 9일 대표직에서 물러난\\xa0이낙연 더불어민주당 대표가 퇴임 기자 간담회에서 한 말이다. 임기 중 가장 후회되는 순간을 묻는 질문 등에 이런 답변이 나왔다.\\xa0\', \' \', \'   \\xa0  \', \'   대표직을 떠난 그는 대선 판도에 작지 않은 영향을 미칠 것으로 예상되는 4·7 재·보궐 선거의 상임선대위원장을 맡게 된다. 그리고 이 선거를 치른 뒤엔 대선 주자로서의 행보를 본격화할 예정이다.\\xa0\', \' \', \'   \\xa0  \', \'   7개월간의 당 대표 기간 동안 그는 한때 40%에 달했던 대선 후보 지지율이 10%대로 곤두박질치는 경험을 했다. \', \'자신에겐 야심작이었을지 몰라도 전직 대통령 사면 건의는 지지율 추락의 주범이었다. \', \'그는 이날 사면 건의에 대해 “언젠가 해야 할 과제라고 생각했었다. 국민 마음을 좀 더 세밀하게 헤아려야 한다는 아픈 공부가 됐다”고 했다. 또 지지율 하락에 대해선 “제 부족함과 정치의 어려움 때문”이라고 말했다.  \', \'   \\xa0  \', \' \', \' \', \' \', \' \', \' \', \' \', \'\', \' \', \'        \\xa0  \', \'   최측근이 스스로 목숨을 끊는 일도 있었다. 이런 우여곡절 끝에 대표직을 떠나는 그에게 서울·부산 보궐선거는 작은 반전의 계기가 될 수도 있다. \', \'오는 5월 전당대회 때까지 김태년 원내대표가 권한대행으로 당을 이끌지만, 이번 재·보선은 상임선대위원장으로 치르는 ‘이낙연의 선거’란 관측이 많다.\', \'\\xa0박원순·오거돈 전직 시장 때문에 치러지는 선거임에도 지난해 11월 당헌·당규 개정과 공천 강행을